In [1]:
# Scraping
from bs4 import BeautifulSoup
import requests

import json
import pandas as pd

import pprint

from urllib.request import Request, urlopen

from time import sleep
from random import randint

In [2]:
# Progress counter/tracker
from contextlib import contextmanager
from timeit import default_timer
import sys
from datetime import timedelta

# Define a timer w/ decoractor to use with the loop
@contextmanager
def elapsed_timer():
    start=default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start

# Testing w/ 2 radiologists

In [3]:
radiologists = ["Richard Duszak", "Andrew Rosenkrantz"]

In [48]:
prefix = 'https://www.doximity.com/pub/'
user_agent = {'User-agent' : 'Mozilla/5.0'}
input_name = radiologists[0].replace(' ', '-').lower()
degree = "-md"
url = prefix+input_name+degree
page = requests.get(url, headers=user_agent)
soup = BeautifulSoup(page.text, "html.parser")

#####################
full_name = soup.find("span", class_="user-full-name").text
first_name = soup.find("span", class_="user-name-first").text
last_name = soup.find("span", class_="user-name-last").text

specialty = soup.find("a", class_="profile-head-subtitle").text
city_state = soup.find("span", class_="profile-head-subtitle").text
subspecialty = soup.find("p", class_="user-subspecialty").text
job_title = soup.find("p", class_="user-job-title").text

st_address = soup.find("span", class_="black profile-contact-labels-wrap").text
PRE_phone_num = soup.find("div", class_="profile-contact-labels-wrap")
phone_num = PRE_phone_num.find("span", class_="black").text

ed_trn_elements = soup.find("ul", class_="profile-sectioned-list training").find_all("div", class_="profile-section-wrapper-text")
for element in ed_trn_elements:
    institution = element.find("span", class_="black").text
    education = element.find("span", class_="br").text
    print(institution)
    print(education)
    print()

########################
# print(full_name)
# print(first_name)
# print(last_name)
# print(specialty)
# print(city_state)
# print(subspecialty)
# print(job_title)
# print(st_address)
# print(phone_num)

University of Pennsylvania Health System
Fellowship, Interventional Radiology - Independent, 1994 - 1995

Duke University Hospital
Residency, Radiology-Diagnostic, 1990 - 1994

York Hospital
Internship, Transitional Year, 1989 - 1990

Pennsylvania State University College of Medicine
Class of 1989

La Salle University
B.A., Biology, Maxima Cum Laude, General University Honors, 1981 - 1985

[<div class="profile-section-wrapper-text"><span class="black" itemprop="name">University of Pennsylvania Health System</span><span class="br">Fellowship, Interventional Radiology - Independent, 1994 - 1995</span></div>, <div class="profile-section-wrapper-text"><span class="black" itemprop="name">Duke University Hospital</span><span class="br">Residency, Radiology-Diagnostic, 1990 - 1994</span></div>, <div class="profile-section-wrapper-text"><span class="black" itemprop="name">York Hospital</span><span class="br">Internship, Transitional Year, 1989 - 1990</span></div>, <div class="profile-section-w

In [45]:
radiologists = ["Richard Duszak", "Andrew Rosenkrantz", "Nishant De Quadros"]
for rad in radiologists:
    print(rad.replace(' ', '-').lower())

richard-duszak
andrew-rosenkrantz
nishant-de-quadros


In [3]:
# Import data
scraper_list = pd.read_csv(r"C:\Users\ssantavicca3\Documents\Work Files & Folders\RadiologyTrainees_NotABot\scraper_list.csv")
scraper_list.head()
for index, rad in scraper_list[0:5].iterrows():
    print(rad["first_nm"])
   

Justin
Matthew
Vincent
Saba
Daniel


In [47]:
# (scraper_list["first_nm"][0]+"-"+scraper_list["last_nm"][0]).lower()
# scraper_list[0:2].iat[0,0]
for index, rad in scraper_list[0:5].iterrows():
    npi = rad["NPI"]
    input_name = (rad["first_nm"]+" "+rad["last_nm"]).replace(' ', '-').lower()
#     input_name = rad.replace(' ', '-').lower()
    print(npi)
    print(input_name)
    

1003002148
justin-owens
1003002783
matthew-hartman
1003002809
vincent-graziano
1003006636
saba-hasan
1003008517
daniel-mendez


In [4]:
# Initialize request details and lists to dump data dicts
prefix = 'https://www.doximity.com/pub/'
user_agent = {'User-agent' : 'Mozilla/5.0'}
verif_data = []
ed_trn_data = []

# Loop through each radiologist and scrape data of interest
with elapsed_timer() as elapsed:
    n_iter = 0
    
    for index, rad in scraper_list.iterrows(): 
        npi = rad["NPI"]
        input_name = (rad["first_nm"]+" "+rad["last_nm"]).replace(' ', '-').lower()  # replace() covers two part last names (e.g., De Soto)
        
        # Beautiful Soup
        try:
            degree = "md"
            url = prefix+input_name+"-"+degree
            page = requests.get(url, headers=user_agent)
        except:
            # Account for DOs, or skip iteration if no hits at all
            try:
                degree = "do"
                url = prefix+input_name+"-"+degree
                page = requests.get(url, headers=user_agent)
            except:
                continue
                
        soup = BeautifulSoup(page.text, "html.parser")

        ## Verification data
        verif_details = {}
        verif_details['NPI'] = npi  #ID
        verif_details['url'] = url

        # Parse out exact text we want & add data to the dictionary
        try:
            verif_details['full_name'] = soup.find("span", class_="user-full-name").text
        except:
            pass

        try:
            verif_details['first_name'] = soup.find("span", class_="user-name-first").text
            verif_details['last_name'] = soup.find("span", class_="user-name-last").text
            verif_details['credentials'] = soup.find("span", class_="user-name-credentials").text
        except:
            pass
        
        try:
            verif_details['specialty'] = soup.find("a", class_="profile-head-subtitle").text
        except:
            pass

        try:
            city_state = soup.find("span", class_="profile-head-subtitle").text
            verif_details['city_state'] = city_state[2:]   #remove "/.n" from beginning of these strings
        except:
            pass

        try:
            verif_details['subspecialty'] = soup.find("p", class_="user-subspecialty").text
        except:
            pass

        try:
            verif_details['job_title'] = soup.find("p", class_="user-job-title").text
        except:
            pass

        try:
            verif_details['st_address'] = soup.find(itemprop="streetAddress").get("content")
            verif_details['zip'] = soup.find(itemprop="postalCode").get("content") 
            verif_details['st_address_full'] = soup.find("span", class_="black profile-contact-labels-wrap").text
        except:
            pass

        try:
            verif_details['phone_num'] = soup.find("div", class_="profile-contact-labels-wrap").find("span", class_="black").text
        except:
            pass
        
        # Append the dictionaries to the list
        verif_details['idx_scraper_list'] = index  
        verif_data.append(verif_details)

        ## Education and training data
        try:
            ed_trn_elements = soup.find("ul", class_="profile-sectioned-list training").find_all("div", class_="profile-section-wrapper-text")
            for element in ed_trn_elements:
                ed_trn_details = {}
                ed_trn_details['NPI'] = npi  #ID

                # Parse out exact text we want & add data to the dictionary
                try:
                    ed_trn_details['institution'] = element.find("span", class_="black").text
                except:
                    pass

                try:
                    ed_trn_details['education'] = element.find("span", class_="br").text
                except:
                    pass

                # Append the dictionaries to the list
                ed_trn_details['idx_scraper_list'] = index
                ed_trn_data.append(ed_trn_details)
        except:
            pass
        
        ## Counter and timer for progress checks  
        n_iter += 1
        if n_iter % 100 == 0:
            print("Iteration (radiologists): "+str(n_iter)+" ----- Time Elapsed: "+str(timedelta(seconds=round(elapsed()))))
            sys.stdout.flush()

            # Save intermediate output incase of crash or timeout
            filename1 = "datadump/doximity dump/saved_ed_trn_list.txt"
            with open(filename1, 'w') as f:
                for item in ed_trn_data:
                    f.write(f'{item}\n')
                    sys.stdout.flush()
            filename2 = "datadump/doximity dump/saved_verif_list.txt"
            with open(filename2, 'w') as f:
                for item in verif_data:
                    f.write(f'{item}\n')
                    sys.stdout.flush()

        # Iteration delay
        sleep(randint(1,2))

Iteration (radiologists): 100 ----- Time Elapsed: 0:02:45
Iteration (radiologists): 200 ----- Time Elapsed: 0:05:26
Iteration (radiologists): 300 ----- Time Elapsed: 0:08:21
Iteration (radiologists): 400 ----- Time Elapsed: 0:11:00
Iteration (radiologists): 500 ----- Time Elapsed: 0:14:05
Iteration (radiologists): 600 ----- Time Elapsed: 0:16:59
Iteration (radiologists): 700 ----- Time Elapsed: 0:19:43
Iteration (radiologists): 800 ----- Time Elapsed: 0:22:26
Iteration (radiologists): 900 ----- Time Elapsed: 0:25:24
Iteration (radiologists): 1000 ----- Time Elapsed: 0:28:02
Iteration (radiologists): 1100 ----- Time Elapsed: 0:30:46
Iteration (radiologists): 1200 ----- Time Elapsed: 0:33:45
Iteration (radiologists): 1300 ----- Time Elapsed: 0:36:35
Iteration (radiologists): 1400 ----- Time Elapsed: 0:39:23
Iteration (radiologists): 1500 ----- Time Elapsed: 0:42:13
Iteration (radiologists): 1600 ----- Time Elapsed: 0:45:13
Iteration (radiologists): 1700 ----- Time Elapsed: 0:48:09
Iterat

Iteration (radiologists): 14000 ----- Time Elapsed: 6:30:17
Iteration (radiologists): 14100 ----- Time Elapsed: 6:33:06
Iteration (radiologists): 14200 ----- Time Elapsed: 6:35:44
Iteration (radiologists): 14300 ----- Time Elapsed: 6:38:51
Iteration (radiologists): 14400 ----- Time Elapsed: 6:41:42
Iteration (radiologists): 14500 ----- Time Elapsed: 6:44:36
Iteration (radiologists): 14600 ----- Time Elapsed: 6:47:22
Iteration (radiologists): 14700 ----- Time Elapsed: 6:50:05
Iteration (radiologists): 14800 ----- Time Elapsed: 6:52:59
Iteration (radiologists): 14900 ----- Time Elapsed: 6:56:06
Iteration (radiologists): 15000 ----- Time Elapsed: 6:58:58
Iteration (radiologists): 15100 ----- Time Elapsed: 7:01:48
Iteration (radiologists): 15200 ----- Time Elapsed: 7:04:37
Iteration (radiologists): 15300 ----- Time Elapsed: 7:07:33
Iteration (radiologists): 15400 ----- Time Elapsed: 7:10:26
Iteration (radiologists): 15500 ----- Time Elapsed: 7:13:14
Iteration (radiologists): 15600 ----- Ti

Iteration (radiologists): 27600 ----- Time Elapsed: 12:48:58
Iteration (radiologists): 27700 ----- Time Elapsed: 12:51:41
Iteration (radiologists): 27800 ----- Time Elapsed: 12:54:28
Iteration (radiologists): 27900 ----- Time Elapsed: 12:57:18
Iteration (radiologists): 28000 ----- Time Elapsed: 12:59:57
Iteration (radiologists): 28100 ----- Time Elapsed: 13:02:47
Iteration (radiologists): 28200 ----- Time Elapsed: 13:05:30
Iteration (radiologists): 28300 ----- Time Elapsed: 13:08:22
Iteration (radiologists): 28400 ----- Time Elapsed: 13:11:06
Iteration (radiologists): 28500 ----- Time Elapsed: 13:13:53
Iteration (radiologists): 28600 ----- Time Elapsed: 13:16:43
Iteration (radiologists): 28700 ----- Time Elapsed: 13:19:29
Iteration (radiologists): 28800 ----- Time Elapsed: 13:22:20
Iteration (radiologists): 28900 ----- Time Elapsed: 13:25:05
Iteration (radiologists): 29000 ----- Time Elapsed: 13:27:59
Iteration (radiologists): 29100 ----- Time Elapsed: 13:30:50
Iteration (radiologists)

In [6]:
## Create and format a DF from the list of dictionaries
ed_trn_df = pd.DataFrame.from_dict(ed_trn_data)
verif_df = pd.DataFrame.from_dict(verif_data)

len(ed_trn_df)
len(verif_df)

NameError: name 'ed_trn_data' is not defined

# Just need to run loop on full list of radiologists. 

## Before doing so, maybe run on first 25-50 rads to (1) get an estimate of ETA, and (2) determine whether there are any bugs that need to be accounted for before full run (e.g., maybe a try/except for MDs/DOs/no page at all).

## May also need to do a follow up scraper with google searches (e.g., "Michael Fishman MD doximity radiology") do get the profiles of people with either NaN or non-radiology specialties that likely shared name with multiple other physicians. Eg:
https://www.doximity.com/pub/michael-fishman-md-ff5f620a
https://www.doximity.com/pub/michael-fishman-md-acf558de
https://www.doximity.com/pub/michael-fishman-md-8b21aefd
## EXPLORE THE MISSING DATA MORE FIRST 
##### i.e., it looks like there are plenty of urls that work but didn't pick up any information (at least in the verif_df file); it even looks like they're missing at consistent intervals??? For the 11,000 or so from the ed_trn_df file, this doesn't appear to be an issue?

In [24]:
# Code to read back in the .txt files created above (i.e., if I close the kernal before saving the local files created above)
from ast import literal_eval

def txt_to_dict(filename):
    tmp_data = list()
    with open(filename, 'r', encoding='cp1252') as f:
        rows = f.readlines()
        for row in rows:
            row = literal_eval(row)
            tmp_data.append(row)
    return tmp_data

In [25]:
ed_trn_dict = txt_to_dict(r"C:\Users\ssantavicca3\Documents\Work Files & Folders\RadiologyTrainees_NotABot\datadump\doximity dump\saved_ed_trn_list.txt")
verif_dict = txt_to_dict(r"C:\Users\ssantavicca3\Documents\Work Files & Folders\RadiologyTrainees_NotABot\datadump\doximity dump\saved_verif_list.txt")

ed_trn_df = pd.DataFrame.from_dict(ed_trn_dict) 
verif_df = pd.DataFrame.from_dict(verif_dict)

ed_trn_df.to_csv(r"C:\Users\ssantavicca3\Documents\Work Files & Folders\RadiologyTrainees_NotABot\ed_trn_df.csv", index=False)
verif_df.to_csv(r"C:\Users\ssantavicca3\Documents\Work Files & Folders\RadiologyTrainees_NotABot\verif_df.csv", index=False)